## Run the STEMMUS_SCOPE model
Steps to run the STEMMUS_SCOPE model, including preprocessing and postprocessing, on Surf super computer Snellius.

In [1]:
import os
from pathlib import Path
import subprocess
from PyStemmusScope import forcing_io 
from PyStemmusScope import iostreamer
from PyStemmusScope import soil_io

#### Path to STEMMUS_SCOPE

STEMMUS_SCOPE source codes are located in the **private** repository on GitHub https://github.com/EcoExtreML/STEMMUS_SCOPE. You need to clone the repository locally and specify the path to it in the cell below. Make sure you have right access to the repository and you created a local branch for development purposes. 

In [2]:
# path to model repository
path_to_model = "path_to_STEMMUS_SCOPE_repository"

#### set git branch and see status

In [3]:
dev_branch = "name_of_your_branch"
!git -C {path_to_model} checkout {dev_branch}
!git -C {path_to_model} status

Already on 'python-compliant'
Your branch is up to date with 'origin/python-compliant'.
On branch python-compliant
Your branch is up to date with 'origin/python-compliant'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	exe/mccExcludedFiles.log
	exe/readme.txt
	exe/requiredMCRProducts.txt
	exe/run_STEMMUS_SCOPE.sh
	exe/unresolvedSymbols.txt
	test_config.txt
	utils/csv_to_nc/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)


#### Update/set config files

In [4]:
# the user must provide the correct path
# path to config file
path_to_config_file = "path_to_config_file"
# Instantiate working directories handler from PyStemmusScope
config_template = iostreamer.read_config(path_to_config_file)

In [5]:
config_template

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '17520',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_2022-05-

In [6]:
# edit config
config_template["NumberOfTimeSteps"] = "5"
config_template

{'WorkDir': '/scratch-shared/ecoextreml/stemmus_scope/',
 'SoilPropertyPath': '/projects/0/einf2480/model_parameters/soil_property/',
 'ForcingPath': '/projects/0/einf2480/forcing/plumber2_data/',
 'ForcingFileName': 'FI-Hyy_1996-2014_FLUXNET2015_Met.nc',
 'directional': '/projects/0/einf2480/model_parameters/vegetation_property/directional/',
 'fluspect_parameters': '/projects/0/einf2480/model_parameters/vegetation_property/fluspect_parameters/',
 'leafangles': '/projects/0/einf2480/model_parameters/vegetation_property/leafangles/',
 'radiationdata': '/projects/0/einf2480/model_parameters/vegetation_property/radiationdata/',
 'soil_spectrum': '/projects/0/einf2480/model_parameters/vegetation_property/soil_spectrum/',
 'input_data': '/projects/0/einf2480/model_parameters/vegetation_property/input_data.xlsx',
 'InitialConditionPath': '/projects/0/einf2480/soil_initialcondition/',
 'NumberOfTimeSteps': '5',
 'InputPath': '/scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_2022-05-11-1

#### Create input directories, prepare input files 

In [7]:
# specify the forcing filename
forcing_filename = config_template["ForcingFileName"]
input_dir, output_dir, config_run_path = iostreamer.create_io_dir(forcing_filename, config_template)
print(input_dir, output_dir, config_run_path)

/scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_20220621_1415 /scratch-shared/ecoextreml/stemmus_scope/output/FI-Hyy_20220621_1415 /scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_20220621_1415/FI-Hyy_20220621_1415_config.txt


In [ ]:
config_run = iostreamer.read_config(config_run_path)
# Generate the forcing input data for STEMMUS_SCOPE
forcing_io.prepare_forcing(input_dir, Path(config_run["ForcingPath"]) / forcing_filename, config_run)
# Generate the soil parameter input data for STEMMUS_SCOPE
soil_io.prepare_soil_data(config_run['SoilPropertyPath'], input_dir, config_run)

#### Run the model

In [8]:
# generate a text file
path_to_code = "path_to_model_src"

# set matlab log dir to slurm, otherwise java.log files are created in user home dir
os.environ['MATLAB_LOG_DIR'] = str(input_dir)

# set matlab arguments
path_to_config = f"'{config_path}'"
command_line = f'matlab -r "STEMMUS_SCOPE_exe({path_to_config});exit;"'
args = [command_line, "-nodisplay", "-nosplash", "-nodesktop"]

# run the model
result = subprocess.run(args, cwd = path_to_code, shell=True)
result.check_returncode()

MATLAB is selecting SOFTWARE OPENGL rendering.
Opening log file:  /scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_20220621_1415/java.log.45704

                            < M A T L A B (R) >
                  Copyright 1984-2021 The MathWorks, Inc.
             R2021a Update 3 (9.10.0.1684407) 64-bit (glnxa64)
                                May 27, 2021

 
To get started, type doc.
For product information, visit www.mathworks.com.
 
Reading config from /scratch-shared/ecoextreml/stemmus_scope/input/FI-Hyy_20220621_1415/FI-Hyy_20220621_1415_config.txt

 The calculations start now 


#### Create output directories, prepare output files

In [10]:
# convert csv files to nc files
path_to_generate_netcdf_script = "path_to_generate_netcdf_script"
path_to_Variables_will_be_in_NetCDF_file = "path_to_Variables_will_be_in_NetCDF_file"
result = subprocess.run(["python", path_to_generate_netcdf_script,
     "--config_file", config_path, "--variable_file",
     path_to_Variables_will_be_in_NetCDF_file])
result.check_returncode()

/scratch-shared/ecoextreml/stemmus_scope/output/FI-Hyy_20220621_1415/ECdata.csv
Reading variable metadata from /home/yangl/STEMMUS_SCOPE/utils/csv_to_nc/Variables_will_be_in_NetCDF_file.csv
Creating /scratch-shared/ecoextreml/stemmus_scope/output/FI-Hyy_20220621_1415/FI-Hyy_20220621_1415_STEMMUS_SCOPE.nc 
Reading data from file: radiation.csv
Reading data from file: fluxes.csv
Reading data from file: surftemp.csv
Reading data from file: Sim_Temp.csv
Reading data from file: Sim_Theta.csv
Reading data from file: aerodyn.csv
Reading data from file: ECdata.csv
Done writing /scratch-shared/ecoextreml/stemmus_scope/output/FI-Hyy_20220621_1415/FI-Hyy_20220621_1415_STEMMUS_SCOPE.nc


#### Generate execuable file

In [11]:
os.environ['MATLAB_LOG_DIR'] = "."
command_line = 'mcc -m ./src/STEMMUS_SCOPE_exe.m -a ./src -d ./exe -o STEMMUS_SCOPE -R nodisplay -R singleCompThread'
result = subprocess.run(command_line, cwd = path_to_model, shell=True)
result.check_returncode()

Opening log file:  ./java.log.52377


#### check changes by git

In [12]:
!git -C {path_to_model} status

On branch python-compliant
Your branch is up to date with 'origin/python-compliant'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   exe/STEMMUS_SCOPE

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	.ipynb_checkpoints/
	exe/mccExcludedFiles.log
	exe/readme.txt
	exe/requiredMCRProducts.txt
	exe/run_STEMMUS_SCOPE.sh
	exe/unresolvedSymbols.txt
	java.log.52377
	test_config.txt
	utils/csv_to_nc/__pycache__/

no changes added to commit (use "git add" and/or "git commit -a")
